In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import shapely
from shapely.geometry import LineString
from matplotlib.path import Path
from shapely import geometry, ops
from scipy.sparse import csr_matrix
import gc
from tqdm import tqdm

### This notebook tells how to generate drivable area masks.

- This is optional.
- We read the prepared dataset from <code>data_generator</code> notebook and run the following blocks sequentially.

In [ ]:
x,y = np.meshgrid(np.arange(-22.5,23,1),np.arange(-11.5, 75, 1))
mesh = np.stack([x.T,y.T],2)
mesh = mesh.reshape(-1, 2)
mesh.shape
def generate_mask(polylines, nbs, width, mesh, y):
    centerlines = np.concatenate([polylines[:nbs,:,:2], polylines[:nbs,-1:,2:4]], 1)
    splines = [LineString(polyline) for polyline in centerlines]
    w = width[:nbs]
    leftBound = [spline.parallel_offset(w_, 'left', resolution=6) for w_, spline in zip(w, splines)]
    rightBound = [spline.parallel_offset(w_, 'right', resolution=6) for w_, spline in zip(w, splines)]
    
    polygons = []
    
    for i in range(len(splines)):
        if leftBound[i].geom_type == 'LineString':
            left = list(leftBound[i].coords)
            print(left)
        else:
            le = [list(element.coords) for element in list(leftBound[i].geoms)]#list(leftBound.geoms)
            left = sum(le, [])#list(itertools.chain.from_iterable(le))
        if rightBound[i].geom_type == 'LineString':
            right = list(rightBound[i].coords)
        else:
            ri = [list(element.coords) for element in list(rightBound[i].geoms)]#list(leftBound.geoms)
            right = sum(ri, [])#list(itertools.chain.from_iterable(ri))
        polygons.append(Path(left+right))
    mask_list = [p.contains_points(mesh) for p in polygons]
    mask = np.any(np.array(mask_list),0)
    mask = mask.reshape(46, 87)
    
    cx, cy = int(y[0]+23), int(y[1]+12)
    if mask[cx, cy]:
        cont = 1
    else:
        cont = 0
    return mask, cont

In [ ]:
'''
This block is for the training set
'''
M = []
N = []
W = []
Y = []
for i in range(1,5):
    data = np.load('./interaction_merge/train'+str(i)+'.npz', allow_pickle=True)
    M.append(data['maps'])
    N.append(data['nbsplines'])
    W.append(data['lanefeature'])
    Y.append(data['intention'])
M = np.concatenate(M,0)
N = np.concatenate(N,0)
W = np.concatenate(W,0)
Y = np.concatenate(Y,0)

S = []
C = []
for i in tqdm(range(len(M))):
    gc.disable()
    s, c = generate_mask(M[i], N[i], W[i].toarray()[...,0], mesh, Y[i]) #this is only for training set and validation set
    S.append(csr_matrix(s))
    C.append(c)
    gc.enable()
S = np.array(S)
C = np.array(C)

np.savez_compressed('./interaction_merge/mask_train', mask=S)

In [ ]:
'''
this is for the validation sets (val/val_all)
'''
data = np.load('./interaction_merge/val.npz', allow_pickle=True) # can be changed to vall_all as well
M = data['maps']
N = data['nbsplines']
W = data['lanefeature']
Y = data['intention']

S = []
C = []
for i in tqdm(range(len(M))):
    gc.disable()
    s, c = generate_mask(M[i], N[i], W[i].toarray()[...,0], mesh, Y[i]) #this is only for training set and validation set
    S.append(csr_matrix(s))
    C.append(c)
    gc.enable()
S = np.array(S)
C = np.array(C)

np.savez_compressed('./interaction_merge/mask_val', mask=S)

In [ ]:
'''
This is for the testing set
'''
data = np.load('./interaction_merge/test.npz', allow_pickle=True) 
M = data['maps']
N = data['nbsplines']
W = data['lanefeature']

S = []
C = []
for i in tqdm(range(len(M))):
    gc.disable()
    s, c = generate_mask(M[i], N[i], W[i].toarray()[...,0], mesh, [0,0])
    S.append(csr_matrix(s))
    C.append(c)
    gc.enable()
S = np.array(S)
C = np.array(C)

np.savez_compressed('./interaction_merge/mask_test', mask=S)